In [24]:
    pip install gradio

Note: you may need to restart the kernel to use updated packages.


In [25]:
import gradio as gr

In [26]:
pip install -r requirements.txt

  Using cached accelerator-2023.7.18.dev1.tar.gz (352 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [2 lines of output]
      Accelerator only works on posix platforms (like Linux).
      You could try running under WSL.
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [27]:
pip install --upgrade accelerate


Note: you may need to restart the kernel to use updated packages.


In [28]:
from accelerate import Accelerator

accelerator = Accelerator()

In [29]:
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                          use_auth_token=True,)

model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                             device_map='auto',
                                             torch_dtype=torch.bfloat16,
                                             use_auth_token=True,
                                            #  load_in_8bit=True,
                                            #  load_in_4bit=True
                                             )

pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 200,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )

llm = HuggingFacePipeline(pipeline = pipe, model_kwargs = {'temperature':0})

c:\Users\abhimava@volvocars.com\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\models\auto\tokenization_auto.py:631: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
c:\Users\abhimava@volvocars.com\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\models\auto\auto_factory.py:460: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 2/2 [00:24<00:00, 12.12s/it]
c:\Users\abhimava@volvocars.com\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\hub.py:373: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [30]:
from langchain.embeddings import HuggingFaceInstructEmbeddings
instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-large",
                                                      model_kwargs={"device": "cpu"})

load INSTRUCTOR_Transformer
max_seq_length  512


In [31]:
from langchain.document_loaders import PyPDFLoader

# Load PDF
loaders = [
    # Duplicate documents on purpose - messy data
    PyPDFLoader("C:/Users/abhimava@volvocars.com/Downloads/Zurich_HackZurich_participant_data/PoliciesEnglish/HouseholdInsurance/GCIHouseholdInsurance.pdf"),
]
docs = []
for loader in loaders:
    docs.extend(loader.load())

In [32]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 3500,
    chunk_overlap = 150
)

In [33]:
splits = text_splitter.split_documents(docs)

In [34]:
from langchain.vectorstores import Chroma

In [35]:
persist_directory = 'insurance_docs/chroma/'

In [36]:
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=instructor_embeddings,
    persist_directory=persist_directory
)

In [37]:
vectordb.persist()

In [38]:
from langchain.chains import RetrievalQA

## Prompt Template

In [42]:
from langchain.prompts import PromptTemplate

In [43]:
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template,)

## Memory

In [44]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

In [50]:
from langchain.chains import ConversationalRetrievalChain
retriever=vectordb.as_retriever()
qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory
)

question = "Can my family benefit and how?"
result = qa({"question": question})
result['answer']

In [68]:
def chat_interface(question):
    if question:
        result = qa({"question": question})
        answer = result['answer']
        return answer

iface = gr.Interface(fn=chat_interface, inputs="text", outputs="text", live=False)

# Start the Gradio interface
iface.launch()

Running on local URL:  http://127.0.0.1:7871

To create a public link, set `share=True` in `launch()`.


In [55]:
question = gr.inputs.Textbox(label = 'Query')
output = gr.outputs.Textbox(label='answer')

C:\Users\abhimava@volvocars.com\AppData\Local\Temp\ipykernel_12072\3670242664.py:1: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  question = gr.inputs.Textbox(label = 'Query')
C:\Users\abhimava@volvocars.com\AppData\Local\Temp\ipykernel_12072\3670242664.py:1: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  question = gr.inputs.Textbox(label = 'Query')
C:\Users\abhimava@volvocars.com\AppData\Local\Temp\ipykernel_12072\3670242664.py:1: GradioDeprecationWarning: `numeric` parameter is deprecated, and it has no effect
  question = gr.inputs.Textbox(label = 'Query')
C:\Users\abhimava@volvocars.com\AppData\Local\Temp\ipykernel_12072\3670242664.py:2: GradioDeprecationWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  output = gr.outputs.Textbox(label='an

In [56]:
iface = gr.Interface(fn=runQuery, inputs="text", outputs="text", live=True)

In [57]:
if __name__ == "__main__":
    iface.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


In [59]:
question = "are accidents covered as well?"
result = qa({"question": question})
result['answer']

'  No, accidents are not covered under this insurance policy. The policy only covers damage caused by fire, sudden and accidental effects of smoke, lightning, explosion, and implosion, as well as damage resulting from extinguishing and clearing work. It does not cover damage caused by accidents, such as car accidents or slips and falls.'